In [ ]:
# Now perform the analysis to generate the first iteration of polygons
for year in range(1980, 1981):

    ### Get the list of netcdf file names to loop through
    if AllOfAustraliaAllTime:
        # Grab everything from the published WOfS all time summaries
        Alltiles = glob.glob(f'{TileFolder}*.nc')
    else:
        Alltiles = Generate_list_of_tile_datasets(ListofAlbersTiles, year,
                                                  TileFolder, CustomData)

    for WOFSfile in Alltiles:
        try:
            # Read in the data
            # Note that the netCDF files we are using here contain a variable called 'frequency',
            # which is what we are using to define our water polygons.
            # If you use a different netCDF input source, you may need to change this variable name here
            WOFSnetCDFData = xr.open_rasterio(f'NETCDF:{WOFSfile}:frequency')
            # Remove the superfluous time dimension
            WOFSnetCDFData = WOFSnetCDFData.squeeze()

#             # Open the clear count variable to generate the minimum observation mask
#             # If you use a different netCDF input source, you may need to change this variable name here
#             WOFSvalidcount = xr.open_rasterio(f'NETCDF:{WOFSfile}:count_clear')
#             WOFSvalidcount = WOFSvalidcount.squeeze()

#             # Filter our WOfS classified data layer to remove noise
#             # Remove any pixels not abserved at least MinimumValidObs times
#             WOFSValidFiltered = WOFSvalidcount >= MinimumValidObs

            for Thresholds in AtLeastThisWet:
                # Remove any pixels that are wet < AtLeastThisWet% of the time
                WOFSfiltered = WOFSnetCDFData > Thresholds

#                 # Now find pixels that meet both the MinimumValidObs and AtLeastThisWet criteria
#                 # Change all zeros to NaN to create a nan/1 mask layer
#                 # Pixels == 1 now represent our water bodies
#                 WOFSfiltered = WOFSfiltered.where((WOFSfiltered != 0) &
#                                                   (WOFSValidFiltered != 0))

                # Convert the raster to polygons
                # We use a mask of '1' to only generate polygons around values of '1' (not NaNs)
                WOFSpolygons = ft.shapes(
                    WOFSfiltered.data.astype('float32'),
                    mask=WOFSfiltered.data.astype('float32') == 1,
                    transform=WOFSnetCDFData.transform)
                # The rasterio.features.shapes returns a tuple. We only want to keep the geometry portion,
                # not the value of each polygon (which here is just 1 for everything)
                WOFSbreaktuple = (a for a, b in WOFSpolygons)

                # Put our polygons into a geopandas geodataframe
                PolygonGP = gp.GeoDataFrame(list(WOFSbreaktuple))

                # Grab the geometries and convert into a shapely geometry
                # so we can quickly calcuate the area of each polygon
                PolygonGP['geometry'] = None
                for ix, poly in PolygonGP.iterrows():
                    poly['geometry'] = shape(poly)

                # Set the geometry of the dataframe to be the shapely geometry we just created
                PolygonGP = PolygonGP.set_geometry('geometry')
                # We need to add the crs back onto the dataframe
                PolygonGP.crs = {'init': 'epsg:3577'}

                # Combine any overlapping polygons
#                 MergedPolygonsGeoms = unary_union(PolygonGP['geometry'])
                MergedPolygonsGPD = PolygonGP

#                 # Turn the combined multipolygon back into a geodataframe
#                 MergedPolygonsGPD = gp.GeoDataFrame(
#                     [poly for poly in MergedPolygonsGeoms])
#                 # Rename the geometry column
#                 MergedPolygonsGPD.columns = ['geometry']
#                 # We need to add the crs back onto the dataframe
#                 MergedPolygonsGPD.crs = {'init': 'epsg:3577'}

                # Calculate the area of each polygon again now that overlapping polygons
                # have been merged
                MergedPolygonsGPD['area'] = MergedPolygonsGPD['geometry'].area

                # Save the polygons to a shapefile
                schema = {
                    'geometry': 'Polygon',
                    'properties': {
                        'area': 'float'
                    }
                }

                # Generate our dynamic filename
                FileName = f'{WaterBodiesShp}_{Thresholds}.shp'
                # Append the file name to the list so we can call it later on

                if os.path.isfile(FileName):
                    with fiona.open(FileName,
                                    "a",
                                    crs=from_epsg(3577),
                                    driver='ESRI Shapefile',
                                    schema=schema) as output:
                        for ix, poly in MergedPolygonsGPD.iterrows():
                            output.write(({
                                'properties': {
                                    'area': poly['area']
                                },
                                'geometry': mapping(shape(poly['geometry']))
                            }))
                else:
                    with fiona.open(FileName,
                                    "w",
                                    crs=from_epsg(3577),
                                    driver='ESRI Shapefile',
                                    schema=schema) as output:
                        for ix, poly in MergedPolygonsGPD.iterrows():
                            output.write(({
                                'properties': {
                                    'area': poly['area']
                                },
                                'geometry': mapping(shape(poly['geometry']))
                            }))

        except:
            print(
                f'{WOFSfile} did not run. \n'
                f'This is probably because there are no waterbodies present in this tile.'
            )